# First script inversion of the ZNS parameters

Thanks to this script we will carry out an inversion of the parameters of the unsaturated area, the porosity and the depth of the water table in steady state from a valerence simulation sampled in 100 points. The heat transfer takes place in a one-dimensional soil of height 1m represented. 

The inversion will focus on the four uncertain parameters and one boundary condition of this model, namely:

- the porosity $n$
- the alpha Van Genuchten parameter $\alpha$,
- the $n$ Van Genuchten parameter, 
- the residual saturation $s_wres$, and
- the water table altitude (WT).


Fill the E_temperature_initiale.dat with the initial temperature profile





# 1 Fill parameters and setup

In [1]:
import os
import numpy as np
from pathlib import Path
import pandas as pd
import array
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import display
import warnings
import math
import sys
import shutil  
import os    
import subprocess
from importlib import reload
# path of the 1D_col directory
os.chdir('/home/ariviere/Programmes/ginette/application/Destin/')

from setup_model import *

import setup_model
reload(setup_model)


libs_gfortran = ['gfortran']
# please compile ginette in the folder 1D_col








/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Parametres

In [2]:
#-----------------------------------------------------------------
########### Setup of the model
#_________________________________________________________________
#time step in 
dt=1
# time reading of the BC in seconds
itlecture=600
#duration of the simulation in days
nb_day=15
# state
## 0 steady state
# 1 transient state (dynamic state)
state=1

# size columnin meter
z_top=169.4

#altitude column bottom in meter
z_bottom=169.4-15

#discretisation : size cell in meter
dz=0.1

#apply unsaturated flow and thermal 
#unsat =1 apply
#unsat=0 cancel unsaturated zone
unsat=1
#_________________________________________________________________

######### PARAMETERS 
#-----------------------------------------------------------------
# number of facies in the column. If nb_zone=1 homognous porous media
nb_zone=3
# constant parameters
## val_k intrinsic permeability [m2]  
## K hydraulic conductivity [m.s-1] k=val_k*mu/(rho*g)
## mu viscosity [Pa.s]
## rho density [kg.m-3]
## g gravity  9.81 [m2.s-1]
# porosity [-]
# nVG van genuchten parameter [-]
# a van genuchten parameter [-]
# swres residual saturation [-]


# Exemple de valeurs pour 3 zones
val_k_list = [1e-13, 5e-13, 1e-13,1e-13]  # Valeurs de k pour chaque zone
val_n_list = [0.35, 0.2, 0.3, 0.3]  # Valeurs de n pour chaque zone
val_a_list = [2, 2, 4, 4]  # Valeurs de a pour chaque zone
val_nVG_list = [2, 2.4, 2, 2]  # Valeurs de nVG pour chaque zone
val_swres_list = [0, 0.1, 0.15*0.3, 0.15*0.3]  # Valeurs de swres pour chaque zone
geom_zone_list=[0.3, 1, 1,11]  # Valeurs de geom pour chaque zone




# files of boundary conditions

In [ ]:
# Boundary condition water level (meter)
#-----------------------------------------------------------------
## z=0 : dirichlet condition when hydraulic head is above z_top and neuman condition when hydraulic head is below z_top
### create boundary condition  hydraulic head increase with sin function in 4 days and decrease in 4 days amplitude 2m
### hydraulic head at the top of the column h_top(t)
### create funtion h_top(t) with a sin function in dataframe



#read data
data_BC=pd.read_csv('data_hydro_P95-B52.txt')

# write E_ec_bc_t.dat with column 1 B52(m) and column 2 0.0
data_BC['bottom']=0.0
data_BC['B52(m)']=data_BC['B52(m)']+z_top
data=data_BC[['B52(m)','bottom']]
data.to_csv('E_ec_bc_t.dat', sep=' ', index=False, header=False)
val_WT=data_BC['P95(m.a.s.l)'][0]
#calculate the time between observations
time_measure=data_BC['minutes'][1]-data_BC['minutes'][0]
print('time between observations',time_measure*60)
#Check the time measure is constant for all data_BC
if all(data_BC['minutes'].diff()[1:]-time_measure==0):
    print('time between observations is constant')
itlecture=int(time_measure*60)
max_time=data_BC['minutes'].max()*60/86400
print('max time',max_time)
nb_day=max_time
# Tracé avec une courbe plus arrondie
plt.plot(data_BC['minutes']*60, data_BC['B52(m)'])
plt.plot(data_BC['minutes']*60, data_BC['P95(m.a.s.l)'])
plt.xlabel('Temps (s)')
plt.ylabel('Hauteur hydraulique (m)')
plt.legend(['B52(m)', 'P95(m.a.s.l)'])
plt.show()
print(val_WT)

# files of boundary conditions

In [ ]:

#-----------------------------------------------------------------
# Compile ginette
if os.path.isfile('ginette'):
    print ("ginette exist")
else:
    print ("ginette not exist")
    print("you must compile ginette in the current directory")
    print(" gfortran -o ginette ../../src/ginette_V2.f")
    subprocess.call(["gfortran","-o","ginette","../../src/ginette_V2.f"])  #creat



nb_cell=domain_state_time(z_top,z_bottom,dz,dt,state,nb_day,unsat)
coord=geometry(z_top,z_bottom,dz,nb_zone)
initial_boundary_condition(val_WT,itlecture)
parameter_zone_geometrie(nb_zone,geom_zone_list,dz,z_top,z_bottom)
# delete the file E_parametre_zone.dat
if os.path.isfile('E_zone_parameter.dat'):
    os.remove('E_zone_parameter.dat')
for i in range(nb_zone):
    parameter_zone_value(i+1, val_k_list[i], val_n_list[i], val_a_list[i], val_nVG_list[i],val_swres_list[i])



## 2. Run the simulation

In [ ]:
# run ginette
subprocess.call(["./ginette"])

# Profile saturation référence au bo

In [ ]:
# read the saturation profil   
saturation_profile_REF = pd.read_table('S_saturation_profil_t.dat',delim_whitespace=True,header=None,names=[ "time",  "z","sat"])
saturation_profile_val=saturation_profile_REF.iloc[saturation_profile_REF.index.stop-int(nb_cell):saturation_profile_REF.index.stop]


In [ ]:
# Trouver les valeurs uniques de temps
unique_times = saturation_profile_REF['time'].unique()

plt.figure(figsize=(10, 6))  # Définir la taille de la figure
plt.style.use('seaborn')  # Utiliser le style seaborn
# Créer un mapping de chaque valeur unique de temps à un indice numérique
time_indices = {time: index for index, time in enumerate(sorted(unique_times))}

plt.figure(figsize=(10, 6))
plt.style.use('seaborn')

# Créer un colormap
cmap = plt.cm.viridis

# Itérer sur chaque valeur unique de temps
for time in unique_times:
    data_for_time = saturation_profile_REF[saturation_profile_REF['time'] == time]
    # Utiliser le colormap pour choisir la couleur en fonction de l'indice du temps
    plt.scatter(data_for_time['sat'], data_for_time['z'], 
                label=f'Time = {time}', 
                c=[cmap(time_indices[time] / len(unique_times))])

plt.xlabel('sat')
plt.ylabel('z')
plt.title('Saturation en fonction de z pour chaque temps')

# Créer une colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min(unique_times), vmax=max(unique_times)))
plt.colorbar(sm, label='Time')

plt.show()

In [ ]:
h_profile_REF = pd.read_table('S_pressure_profil_t.dat',delim_whitespace=True,header=None,names=[ "time",  "z","pressure","h"])
#hydraulic head for the min z 

min_z=z_top-1
# find nearest value to min_z

h_profile_val=h_profile_REF[(h_profile_REF['z']<=min_z+dz)&(h_profile_REF['z']>=min_z-dz)]
plt.figure()
plt.style.use('seaborn')
plt.scatter(h_profile_val.time,h_profile_val.h, s=10, alpha=1, color='mediumblue',marker='.')
plt.scatter(data_BC['minutes']*60, data_BC['P95(m.a.s.l)'], s=10, alpha=1, color='red',marker='.')